# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission, if necessary. Sections that begin with **'Implementation'** in the header indicate where you should begin your implementation for your project. Note that some sections of implementation are optional, and will be marked with **'Optional'** in the header.

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [1]:
# Load pickled data
import pickle

# TODO: Fill this in based on where you saved the training and testing data

import os
import zipfile
import urllib.request

training_file = './data/train.p'
testing_file = './data/test.p'
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2016/November/581faac4_traffic-signs-data/traffic-signs-data.zip'

if not os.path.isfile(training_file):
    data_dir = os.path.dirname(training_file)
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    zip_file = os.path.join(data_dir, 'traffic-signs-data.zip')
    
    print('Downloading file...')
    urllib.request.urlretrieve(url, zip_file)
    
    print('Unzipping file...')
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(data_dir)
        
    print('Done')
        
assert(os.path.isfile(training_file))
assert(os.path.isfile(testing_file))

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 2D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below.

In [2]:
print(len(X_train[0]))
print(len(X_train[0][0]))
print(y_train)

32
32
[ 0  0  0 ..., 42 42 42]


In [3]:
### Replace each question mark with the appropriate value.

# TODO: Number of training examples
n_train = len(X_train)

# TODO: Number of testing examples.
n_test = len(X_test)

# TODO: What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc.

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections.

In [4]:
### Data exploration visualization goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [5]:
from IPython.display import display as display
from IPython.core.pylabtools import figsize, getfigs
import math
import matplotlib.image as mpimg
import matplotlib.transforms as mtransforms
import numpy as np
import cv2

In [6]:
with open('./signnames.csv', 'r') as signnames:
    class_defs = [line.strip().split(',') for line in signnames]
    
class_def_map = dict(class_defs)

In [7]:
def draw_multiple(images, title, cols = 2):
    """ Draw multiple images with a title. """
    plt.close('all')
    plt.axis('off')
    rows = math.ceil(len(images)/cols)
    f, axarr = plt.subplots(rows, cols, sharex='col', sharey='row')
    DPI = f.get_dpi() / 4
    f.set_size_inches(images[0].shape[1]*cols/float(DPI), images[0].shape[0]*rows/float(DPI))
    for i in range(len(images)):
        col = i % cols
        row = math.floor(i / cols)
        axarr[row, col].imshow(images[i], cmap='gray')
        axarr[row, col].axis('off')
    f.suptitle(title)
    f.subplots_adjust(hspace=0)
    display(f)
    plt.close(f)
    
def draw(img, title):
    """ Draw a single image with a title """
    f = plt.figure()
    plt.title(title)
    plt.imshow(img, cmap='gray')
    display(f)
    plt.close(f)


In [8]:
from textwrap import wrap

def get_n_random_images_with_class_c(n, c):
    indices = [a for a in range(len(y_train)) if y_train[a] == c]
    selected_indices = np.random.choice(indices, size=(n,))
    selected_images = X_train[selected_indices,:,:,:]
    return selected_images

def draw_n_images_of_each_class(n):
    plt.close('all')
    plt.axis('off')
    cols = n
    rows = n_classes
    images = []
    for i in range(n_classes):
        images.extend(get_n_random_images_with_class_c(n, i))
    f, axarr = plt.subplots(rows, cols, sharex='col', sharey='row')
    DPI = f.get_dpi()
    f.set_size_inches(64*cols/float(DPI) + 3 , 64*rows/float(DPI))
    plt.setp(axarr.flat, xticks=[], yticks=[])
    for i in range(len(images)):
        col = i % cols
        row = math.floor(i / cols)
        axarr[row, col].imshow(images[i], cmap='gray')
        axarr[row, col].spines['top'].set_visible(False)
        axarr[row, col].spines['right'].set_visible(False)
        axarr[row, col].spines['bottom'].set_visible(False)
        axarr[row, col].spines['left'].set_visible(False)
    for ax, ve in zip(axarr[0], range(n)):
        ax.set_title('{0}'.format(ve))
    for ax, c in zip(axarr[:, 0], range(n_classes)):
        label = '{0} : {1}'.format(c, class_def_map[str(c)])
        ax.set_ylabel('\n'.join(wrap(label, 20)), rotation = 0, horizontalalignment='right', verticalalignment='center')
    f.subplots_adjust(hspace=0.001, wspace= 0.001)
    plt.tight_layout()
    f.subplots_adjust(hspace=0.001, wspace= 0.001)

    display(f)
    plt.close(f)
    
#draw_n_images_of_each_class(5)

In [9]:
import random
import numpy as np

def get_random_image():
  index = random.randint(0, len(X_train))
  image = X_train[index].squeeze()
  return image

def scale(img, scale):
    return rotate(img, 0, scale)

def rotate(img, angle, scale = 1.0):
    center = tuple(np.array(img.shape[0:2])/2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, scale)
    return cv2.warpAffine(img, rotation_matrix, img.shape[0:2])

def translate(img, x, y):
    M = np.float32([[1, 0, x], [0, 1, y]])
    return cv2.warpAffine(img, M, img.shape[0:2])

def shear(img, x, y):
    M = np.float32([[1, math.tan(x), 0], [math.tan(y), 1, 0]])
    return cv2.warpAffine(img, M, img.shape[0:2])

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4,4))
def clahe_l(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

def clahe_rgb(img):
    r, g, b = cv2.split(img)
    cr = clahe.apply(r)
    cg = clahe.apply(g)
    cb = clahe.apply(b)
    final = cv2.merge((cr, cg, cb))
    return final

def draw_polygon(img, pts, color, alpha):
    pts = pts.reshape((-1, 1, 2))
    hull = cv2.convexHull(pts)
    copy = img.copy()
    cv2.fillPoly(copy, [hull], color)
    return cv2.addWeighted(copy, alpha, img, 1-alpha, 0)

def hls_transform(img, h, l, s):
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    hls[..., 0] = (hls[..., 0]+h)
    hls[..., 1] = (hls[..., 1]+l)
    hls[..., 2] = (hls[..., 2]+s)
    rgb = cv2.cvtColor(hls, cv2.COLOR_HLS2BGR)
    return rgb

def motion_blur(img, amount):
    kernel = np.zeros((amount, amount))
    kernel[int((amount-1)/2), :] = np.ones(amount)
    kernel = kernel / amount
    return cv2.filter2D(img, -1, kernel)

def change_brightness(img, amount):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv[:,:,2] += amount
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def adjust_gamma(img, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table)

SCALE_PROB = 0.15
ROTATE_PROB = 0.15
TRANSLATE_PROB = 0.15
SHEAR_PROB = 0.1
POLYGON_PROB = 0.10
HLS_PROB = 0.0
BLUR_PROB = 0.05
CHANGE_BRIGHTNESS_PROB = 0.2
def augment(img):
    output = img
    if np.random.rand() < SCALE_PROB:
        output = scale(output, random.uniform(0.8, 1.2))
    if np.random.rand() < ROTATE_PROB:
        output = rotate(output, np.deg2rad(random.uniform(-10, 10)))
    if np.random.rand() < TRANSLATE_PROB:
        output = translate(output, random.randint(-3, 3), random.randint(-3, 3))
    if np.random.rand() < SHEAR_PROB:
        output = shear(output, np.deg2rad(random.uniform(-12, 12)), np.deg2rad(random.uniform(-12, 12)))
    if np.random.rand() < POLYGON_PROB:
        color = (10, 10, 10)
        pts = np.array([[random.randint(0, 32), random.randint(0, 32)], [random.randint(0, 32), random.randint(0, 32)],
                        [random.randint(0, 32), random.randint(0, 32)], [random.randint(0, 32), random.randint(0, 32)]],
                        np.int32)
        output = draw_polygon(output, pts, color,
                              random.uniform(0.1, 0.8))
    if np.random.rand() < HLS_PROB:
        output = hls_transform(output, random.randint(0, 10), random.randint(0, 10), random.randint(0, 10))
    if np.random.rand() < BLUR_PROB:
        output = motion_blur(output, random.randint(1, 5))
    if np.random.rand() < CHANGE_BRIGHTNESS_PROB:
        output = adjust_gamma(output, random.uniform(0.8, 3.0))
    return output

def repeat_augment(img, n):
    output = img
    for i in range(n):
        output = augment(output)
    return output

def augment_multiple(img, repeats, n):
    return [repeat_augment(img, repeats) for _ in range(n)]

def augment_multiple_preprocess(img, repeats, n):
    return [preprocess(repeat_augment(img, repeats)) for _ in range(n)]

def preprocess(img):
    return clahe_rgb(img)

def draw_n_augmented_images_of_each_class(n, x):
    plt.close('all')
    plt.axis('off')
    cols = n + x
    rows = n_classes
    images = []
    for i in range(n_classes):
        images.extend([preprocess(img) for img in get_n_random_images_with_class_c(x, i)])
        image = get_n_random_images_with_class_c(1, i)[0]
        augmented_images = augment_multiple_preprocess(image, 5, n - 1)
        images.extend([preprocess(image)])
        images.extend(augmented_images)

    f, axarr = plt.subplots(rows, cols, sharex='col', sharey='row')
    DPI = f.get_dpi()
    f.set_size_inches(64*cols/float(DPI) + 3 , 64*rows/float(DPI))
    plt.setp(axarr.flat, xticks=[], yticks=[])
    for i in range(len(images)):
        col = i % cols
        row = math.floor(i / cols)
        axarr[row, col].imshow(images[i], cmap='gray')
        axarr[row, col].spines['top'].set_visible(False)
        axarr[row, col].spines['right'].set_visible(False)
        axarr[row, col].spines['bottom'].set_visible(False)
        axarr[row, col].spines['left'].set_visible(False)
    for ax, ve in zip(axarr[0], range(cols)):
        label = "Base" if ve < x else 'Aug {0}'.format(ve-x)
        ax.set_title(label)
    for ax, c in zip(axarr[:, 0], range(n_classes)):
        label = '{0} : {1}'.format(c, class_def_map[str(c)])
        ax.set_ylabel('\n'.join(wrap(label, 20)), rotation = 0, horizontalalignment='right', verticalalignment='center')
    f.subplots_adjust(hspace=0.001, wspace= 0.001)
    plt.tight_layout()
    f.subplots_adjust(hspace=0.001, wspace= 0.001)

    display(f)
    plt.close(f)
    
#draw_n_augmented_images_of_each_class(5, 3)
#test_image = get_random_image()

#draw(preprocess(test_image), "Original")
#draw_multiple(augment_multiple_preprocess(test_image, 5, 10), "Augmented")




----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

There are various aspects to consider when thinking about this problem:

- Neural network architecture
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

**NOTE:** The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play!

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [10]:
### Preprocess the data here.
### Feel free to use as many code cells as needed.

In [11]:
#from sklearn.utils import shuffle
#from sklearn.model_selection import train_test_split

#X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train)
#X_train, y_train = shuffle(X_train, y_train)

#print(len(X_train))
#print(len(X_validation))

In [12]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

NUM_AUGMENT = 7

from sklearn.utils import shuffle, resample
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

X_train = np.vstack((X_train, np.vstack(augment_multiple(x, 2, NUM_AUGMENT) for x in X_train)))
y_train = np.hstack((y_train, np.repeat(y_train, NUM_AUGMENT, axis=0)))

X_train = np.float32([preprocess(x) for x in X_train])
X_val = np.float32([preprocess(x) for x in X_val])
X_test = np.float32([preprocess(x) for x in X_test])

X_validation = X_val
y_validation = y_val

### Question 1 

_Describe how you preprocessed the data. Why did you choose that technique?_

**Answer:**

In [13]:
### Generate data additional data (OPTIONAL!)
### and split the data into training/validation/testing sets here.
### Feel free to use as many code cells as needed.

### Question 2

_Describe how you set up the training, validation and testing data for your model. **Optional**: If you generated additional data, how did you generate the data? Why did you generate the data? What are the differences in the new dataset (with generated data) from the original dataset?_

**Answer:**

In [14]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

In [15]:
import os

import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.examples.tutorials.mnist import mnist
EPOCHS = 25
BATCH_SIZE = 256
DROPOUT_KEEP_PROB = 0.7

dropout_prob = tf.placeholder(tf.float32)
def foonet(images, is_training=True):
    net = slim.layers.conv2d(images, 8, [3,3], scope='fc1a')
    net = slim.layers.conv2d(images, 16, [3,3], scope='fc1b')
    net = slim.layers.max_pool2d(net, 2, 2, scope='fp1')
    net = slim.dropout(net, dropout_prob, is_training=is_training, scope='fdropout1')
    net = slim.layers.conv2d(net, 32, [3,3], scope='fc2a')
    net = slim.layers.conv2d(net, 64, [3,3], scope='fc2b')
    #Testing this:
    b1 = slim.layers.conv2d(net, 128, [3,3], scope='fc2c')
    b1 = slim.layers.max_pool2d(b1, 2, 2, scope='fp2')    
    #net = slim.layers.max_pool2d(net, [2,2], 2, scope='p2')
    b1 = slim.dropout(b1, dropout_prob, is_training=is_training, scope='fdropout2')    
    b1 = slim.layers.flatten(b1, scope='ff3') 
    
    b2 = slim.repeat(net, 2, slim.conv2d, 64, [3, 3], scope='fconv1')
    b2 = slim.max_pool2d(b2, [2, 2], scope='fpool1')
    b2 = slim.repeat(b2, 2, slim.conv2d, 128, [3, 3], scope='fconv2')
    b2 = slim.max_pool2d(b2, [2, 2], scope='fpool2')
    b2 = slim.repeat(b2, 4, slim.conv2d, 256, [3, 3], scope='fconv3')
    b2 = slim.max_pool2d(b2, [2, 2], scope='fpool3')
    b2 = slim.repeat(b2, 4, slim.conv2d, 512, [3, 3], scope='fconv4')
    b2 = slim.max_pool2d(b2, [2, 2], scope='fpool4')
    b2 = slim.repeat(b2, 4, slim.conv2d, 512, [3, 3], scope='fconv5')
    #b2 = slim.max_pool2d(b2, [2, 2], scope='fpool5')
    b2 = slim.layers.flatten(b2, scope='ffx')
    # Use conv2d instead of fully_connected layers.
    b2 = slim.layers.fully_connected(b2, 4096, scope='ffc6')
    
    b1 = slim.layers.fully_connected(b1, 1024, scope='ffully_connected4')
    net = tf.concat([b1, b2], 3)
    net = slim.dropout(net, dropout_prob, is_training=is_training, scope='fdrouput4')
    net = slim.layers.fully_connected(net, 43, activation_fn=None, scope='ffully_connected5')
    return net

def lenet(images, is_training=True):
   net = slim.layers.conv2d(images, 8, [3,3], scope='c1a')
   net = slim.layers.conv2d(images, 16, [3,3], scope='c1b')
   net = slim.layers.max_pool2d(net, 2, 2, scope='p1')
   net = slim.dropout(net, dropout_prob, is_training=is_training, scope='dropout1')
   net = slim.layers.conv2d(net, 32, [3,3], scope='c2a')
   net = slim.layers.conv2d(net, 64, [3,3], scope='c2b')
   #Testing this:
   net = slim.layers.conv2d(net, 128, [3,3], scope='c2c')
   net = slim.layers.max_pool2d(net, 2, 2, scope='p2')    
   #net = slim.layers.max_pool2d(net, [2,2], 2, scope='p2')
   net = slim.dropout(net, dropout_prob, is_training=is_training, scope='dropout2')    
   net = slim.layers.flatten(net, scope='f3')
   net = slim.layers.fully_connected(net, 1024, scope='fully_connected4')
   net = slim.dropout(net, dropout_prob, is_training=is_training, scope='drouput4')
   net = slim.layers.fully_connected(net, 43, activation_fn=None, scope='fully_connected5')
   return net

def vgg_19(inputs, num_classes=43, dropout_keep_prob=DROPOUT_KEEP_PROB, is_training=True, spatial_squeeze=True, scope='vgg_19'):
  with tf.variable_scope(scope, 'vgg_19', [inputs]) as sc:
    end_points_collection = sc.name + '_end_points'
    # Collect outputs for conv2d, fully_connected and max_pool2d.
    with slim.arg_scope([slim.conv2d, slim.fully_connected, slim.max_pool2d],
                        outputs_collections=end_points_collection):
      net = slim.repeat(inputs, 2, slim.conv2d, 64, [3, 3], scope='conv1')
      net = slim.max_pool2d(net, [2, 2], scope='pool1')
      net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3], scope='conv2')
      net = slim.max_pool2d(net, [2, 2], scope='pool2')
      net = slim.repeat(net, 4, slim.conv2d, 256, [3, 3], scope='conv3')
      net = slim.max_pool2d(net, [2, 2], scope='pool3')
      net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv4')
      net = slim.max_pool2d(net, [2, 2], scope='pool4')
      net = slim.repeat(net, 4, slim.conv2d, 512, [3, 3], scope='conv5')
      net = slim.max_pool2d(net, [2, 2], scope='pool5')
      # Use conv2d instead of fully_connected layers.
      net = slim.layers.fully_connected(net, 4096, scope='fc6')
      #net = slim.conv2d(net, 4096, [7, 7], padding='VALID', scope='fc6')
      net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout6')
      net = slim.conv2d(net, 4096, [1, 1], scope='fc7')
      net = slim.dropout(net, dropout_keep_prob, is_training=is_training,
                         scope='dropout7')
      net = slim.conv2d(net, num_classes, [1, 1],
                        activation_fn=None,
                        normalizer_fn=None,
                        scope='fc8')
      # Convert end_points_collection into a end_point dict.
      end_points = slim.utils.convert_collection_to_dict(end_points_collection)
      if spatial_squeeze:
        net = tf.squeeze(net, [1, 2], name='fc8/squeezed')
        end_points[sc.name + '/fc8'] = net
      return net
vgg_19.default_image_size = 32

x = tf.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

### Question 3

_What does your final architecture look like? (Type of model, layers, sizes, connectivity, etc.)  For reference on how to build a deep neural network using TensorFlow, see [Deep Neural Network in TensorFlow
](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/b516a270-8600-4f93-a0a3-20dfeabe5da6/concepts/83a3a2a2-a9bd-4b7b-95b0-eb924ab14432) from the classroom._


**Answer:**

In [16]:
### Train your model here.
### Feel free to use as many code cells as needed.

In [17]:
rate = 0.001

logits = lenet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, one_hot_y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [18]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, dropout_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [19]:
from datetime import datetime
print(datetime.now().time())
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y, dropout_prob: DROPOUT_KEEP_PROB})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        print("TIME {} \t EPOCH {} \t Validation Accuracy = {:.6f}".format(datetime.now().time(), i+1, validation_accuracy))
        
    saver.save(sess, 'lenet')
    print("Model saved")

01:49:50.547990
Training...

TIME 01:51:39.624289 	 EPOCH 1 	 Validation Accuracy = 0.496073
TIME 01:53:26.446092 	 EPOCH 2 	 Validation Accuracy = 0.809344
TIME 01:55:13.354429 	 EPOCH 3 	 Validation Accuracy = 0.977966
TIME 01:57:00.249255 	 EPOCH 4 	 Validation Accuracy = 0.989697
TIME 01:58:47.118328 	 EPOCH 5 	 Validation Accuracy = 0.993777
TIME 02:00:34.110595 	 EPOCH 6 	 Validation Accuracy = 0.994287
TIME 02:02:21.001242 	 EPOCH 7 	 Validation Accuracy = 0.995920
TIME 02:04:07.930299 	 EPOCH 8 	 Validation Accuracy = 0.996838
TIME 02:05:54.881216 	 EPOCH 9 	 Validation Accuracy = 0.996532
TIME 02:07:41.732023 	 EPOCH 10 	 Validation Accuracy = 0.996226
TIME 02:09:28.630134 	 EPOCH 11 	 Validation Accuracy = 0.998164
TIME 02:11:15.521257 	 EPOCH 12 	 Validation Accuracy = 0.997552
TIME 02:13:02.360464 	 EPOCH 13 	 Validation Accuracy = 0.997858
TIME 02:14:49.278329 	 EPOCH 14 	 Validation Accuracy = 0.997450
TIME 02:16:36.126090 	 EPOCH 15 	 Validation Accuracy = 0.997654
TIME 

### Question 4

_How did you train your model? (Type of optimizer, batch size, epochs, hyperparameters, etc.)_


**Answer:**

### Question 5


_What approach did you take in coming up with a solution to this problem? It may have been a process of trial and error, in which case, outline the steps you took to get to the final solution and why you chose those steps. Perhaps your solution involved an already well known implementation or architecture. In this case, discuss why you think this is suitable for the current problem._

**Answer:**

---

## Step 3: Test a Model on New Images

Take several pictures of traffic signs that you find on the web or around you (at least five), and run them through your classifier on your computer to produce example results. The classifier might not recognize some local signs but it could prove interesting nonetheless.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Implementation

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project. Once you have completed your implementation and are satisfied with the results, be sure to thoroughly answer the questions that follow.

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Question 6

_Choose five candidate images of traffic signs and provide them in the report. Are there any particular qualities of the image(s) that might make classification difficult? It could be helpful to plot the images in the notebook._



**Answer:**

In [ ]:
### Run the predictions here.
### Feel free to use as many code cells as needed.

### Question 7

_Is your model able to perform equally well on captured pictures when compared to testing on the dataset? The simplest way to do this check the accuracy of the predictions. For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate._

_**NOTE:** You could check the accuracy manually by using `signnames.csv` (same directory). This file has a mapping from the class id (0-42) to the corresponding sign name. So, you could take the class id the model outputs, lookup the name in `signnames.csv` and see if it matches the sign from the image._


**Answer:**

In [ ]:
### Visualize the softmax probabilities here.
### Feel free to use as many code cells as needed.

### Question 8

*Use the model's softmax probabilities to visualize the **certainty** of its predictions, [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. Which predictions is the model certain of? Uncertain? If the model was incorrect in its initial prediction, does the correct prediction appear in the top k? (k should be 5 at most)*

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

**Answer:**

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.